In [1]:
import sys
import tensorflow as tf
tf.__version__

/home/zzn/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.12.0'

In [ ]:
import ADCrowdNet
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys
%matplotlib inline
import DME_deformable
result_output = open("/home/zzn/SANet_implementation-master/result_B_12.13.txt", "w")
image_train_path = "/home/zzn/part_B_final/train_data/images_train.npy"
gt_train_path = "/home/zzn/part_B_final/train_data/gt_train.npy"
image_validate_path = "/home/zzn/part_B_final/train_data/images_validate.npy"
gt_validate_path = "/home/zzn/part_B_final/train_data/gt_validate.npy"
batch_size = 1
epoch = 500
MAE = 19970305
if __name__ == '__main__':
    image_train = np.load(image_train_path)
    gt_train = np.load(gt_train_path)
    image_validate = np.load(image_validate_path)
    gt_validate = np.load(gt_validate_path)
    
    x = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="input")
    y = tf.placeholder(tf.float32, shape=[None, None, None, 1], name="label")
    
    estimated_density_map = DME_deformable.DME_model(x, 1, 384, 512)
    
    estimated_counting = tf.reduce_sum(estimated_density_map, reduction_indices=[1, 2, 3], name='estimated_counting')
    gt_counting = tf.cast(tf.reduce_sum(y, reduction_indices=[1, 2, 3]), tf.float32)
    
    sum_filter = tf.constant([1., 1., 1., 1.], dtype=tf.float32, shape=[2, 2, 1, 1])
    gt_map = tf.nn.conv2d(y, sum_filter, [1, 2, 2, 1], padding='SAME')
    
    loss = tf.losses.mean_squared_error(gt_map, predictions=estimated_density_map)
    train_op = tf.train.AdamOptimizer(1e-5).minimize(loss=loss, global_step=tf.train.get_global_step())
    
    AE_batch = tf.abs(tf.subtract(estimated_counting, gt_counting))
    SE_batch = tf.square(tf.subtract(gt_counting, estimated_counting))
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # init the Variables
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        image_train_num = len(image_train)
        step = 0
        for i in range(epoch):
            shuffle_batch = np.random.permutation(image_train_num // batch_size)
            for j in range(image_train_num // batch_size):
                # train
                start = (shuffle_batch[j] * batch_size) % image_train_num
                end = min(start + batch_size, image_train_num)
                sess.run(train_op, feed_dict={x: image_train[start:end], y: gt_train[start:end]})
                step = step + 1
                
                #validate
                if step%50 == 0:
                    loss_ = []
                    MAE_ = []
                    MSE_ = []
                    for k in range(len(image_validate // batch_size)):
                        loss_eval, batch_average_error, batch_square_error = sess.run([loss, AE_batch, SE_batch], feed_dict={x: image_validate[k:k+batch_size], y: gt_validate[k:k+batch_size]})
                        loss_.append(loss_eval)
                        MAE_.append(batch_average_error)
                        MSE_.append(batch_square_error)
                    
                    loss_ = np.reshape(loss_, [-1])
                    MAE_ = np.reshape(MAE_, [-1])
                    MSE_ = np.reshape(MSE_, [-1])
                    
                    # calculate the validate loss, validate MAE and validate RMSE
                    validate_loss = np.mean(loss_)
                    validate_MAE = np.mean(MAE_)
                    validate_RMSE = np.sqrt(np.mean(MSE_))
                    
                     # show one of the validate samples
                    figure, (origin, density_gt, pred) = plt.subplots(1, 3, figsize=(20, 4))
                    origin.imshow(image_validate[1])
                    origin.set_title('Origin Image')
                    gt_validate_down_sampling_map, predict_den, gt_counts, pred_counts = sess.run([gt_map, estimated_density_map, gt_counting, estimated_counting], feed_dict={x: image_validate[1:2], y: gt_validate[1:2]})
                    density_gt.imshow(np.squeeze(gt_validate_down_sampling_map), cmap=plt.cm.jet)
                    density_gt.set_title('ground_truth')
                    
                    predict_den = np.squeeze(predict_den)
                    pred.imshow(predict_den, cmap = plt.cm.jet)
                    plt.suptitle("one sample from the validate")
                    plt.show()
                    
                    # show the validate MAE and MSE values on stdout
                    gt_counts = np.squeeze(gt_counts)
                    pred_counts = np.squeeze(pred_counts)
                    sys.stdout.write('The gt counts of the above sample:{}, and the pred counts:{}\n'.format(gt_counts, pred_counts))
                    sys.stdout.write('In step {}, epoch {}, with loss {}, MAE = {}, MSE = {}\n'.format(step, i + 1, validate_loss, validate_MAE, validate_RMSE))            
                    sys.stdout.flush()
                    
                    # save model
                    if MAE > validate_MAE:
                        MAE = validate_MAE
                        saver.save(sess, './checkpoint_dir/MyModel_deformable')

In [1]:


inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
front_end, init_fn = ADCrowdNet.ADCrowdNet_model(inputs)
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
init_fn(sess)

NameError: name 'tf' is not defined

In [20]:


with tf.Session() as sess:
    w1 = tf.constant([[1., 2.], [3., 4.]])
    w2 = tf.constant([[[[1., 2., 3.] for i in range(3)] for j in range(3)], [[[1., 2., 3.] for i in range(3)] for j in range(3)]])
#     w2= tf.split(w2, 2, 0)
#     w3 = tf.nn.softmax(w1, 1)
#     w1 = tf.split(w1, 2, 0)
#     lit = []
#     for i, j in zip(w1, w2):
#         i = tf.squeeze(i)
#         fb, fc = tf.split(j, 2, 3)
#         lit.append(i[0] * fb + i[1] * fc)
#     print(sess.run(w3))
#     print(sess.run(tf.concat(lit, 0)))
#     lit = []
#     for i, j in zip(w1, w2):
#         fb, fc = tf.split(j, 2, axis=3)
#         lit.append(i[0] * fb + i[1] * fc)
    offset = tf.slice(w2, [0, 0, 0, 0], [2, 3, 3, 2])
    mask = tf.slice(w2, [0, 0, 0, 2], [2, 3, 3, 1])
    print(sess.run(offset))
    print(sess.run(mask))

[[[[1. 2.]
   [1. 2.]
   [1. 2.]]

  [[1. 2.]
   [1. 2.]
   [1. 2.]]

  [[1. 2.]
   [1. 2.]
   [1. 2.]]]


 [[[1. 2.]
   [1. 2.]
   [1. 2.]]

  [[1. 2.]
   [1. 2.]
   [1. 2.]]

  [[1. 2.]
   [1. 2.]
   [1. 2.]]]]
[[[[3.]
   [3.]
   [3.]]

  [[3.]
   [3.]
   [3.]]

  [[3.]
   [3.]
   [3.]]]


 [[[3.]
   [3.]
   [3.]]

  [[3.]
   [3.]
   [3.]]

  [[3.]
   [3.]
   [3.]]]]


In [21]:
import tensorflow as tf
in_h, in_w = [int(i) for i in [11, 8]]
print(in_h, in_w)
feat_h, feat_w = [int(i) for i in [in_h, in_w]]
x, y = tf.meshgrid(tf.range(feat_w), tf.range(feat_h))
x, y = [tf.reshape(i, [1, *i.get_shape(), 1]) for i in [x, y]]  # shape [1, h, w, 1]
x, y = [tf.image.extract_image_patches(i, [1, 3, 3, 1], [1, 1, 1, 1], [1, 1, 1, 1], 'VALID') 
        for i in [x, y]] 
sess = tf.Session()
a, b = sess.run([x, y])
for i in [a, b]:
    print(i)

11 8
[[[[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2 0 1 2]
   [1 2 3 1 2 3 1 2 3]
   [2 3 4 2 3 4 2 3 4]
   [3 4 5 3 4 5 3 4 5]
   [4 5 6 4 5 6 4 5 6]
   [5 6 7 5 6 7 5 6 7]]

  [[0 1 2 0 1 2

In [2]:
import tensorflow as tf
sum_filter = tf.constant([1., 1., 1., 1.], dtype=tf.float32, shape=[2, 2, 1, 1])
with tf.Session() as sess:
    print(sess.run(a))

[[[[1.]]

  [[1.]]]


 [[[1.]]

  [[1.]]]]
